In [1]:
pip install ezdxf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 68.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import ezdxf

# Load DXF file
doc = ezdxf.readfile("/kaggle/input/intersection-one/59.Intersection drawings of highways Autocad File DWG.dxf")
msp = doc.modelspace()

# Extract entities (lines, polylines, arcs, layers, etc.)
entities = []
for entity in msp:
    if entity.dxftype() in ["LINE", "LWPOLYLINE", "ARC"]:
        ent_dict = {
            "type": entity.dxftype(),
            "layer": entity.dxf.layer,
            "points": [],         # stores coordinates
            "meta": {}            # additional attributes (radii, etc.)
        }
        if entity.dxftype() == "LINE":
            ent_dict["points"] = [entity.dxf.start, entity.dxf.end]
        elif entity.dxftype() == "LWPOLYLINE":
            ent_dict["points"] = [point for point in entity.get_points()]
        elif entity.dxftype() == "ARC":
            ent_dict["points"] = [entity.dxf.center]
            ent_dict["meta"]["radius"] = entity.dxf.radius
        entities.append(ent_dict)


In [3]:
print(entities)

[{'type': 'LWPOLYLINE', 'layer': 'Bariyer orta', 'points': [(461964.6917797081, 3583669.059972944, 0.0, 0.0, 0.0157131495682483), (461808.7852239612, 3583800.965876329, 0.0, 0.0, 0.0), (460389.4053817891, 3584927.271769748, 0.0, 0.0, 0.0)], 'meta': {}}, {'type': 'LWPOLYLINE', 'layer': 'Bariyer orta', 'points': [(460389.405381789, 3584927.271769748, 0.0, 0.0, 0.0), (460372.6995583878, 3584940.528170037, 0.0, 0.0, 0.0)], 'meta': {}}, {'type': 'LWPOLYLINE', 'layer': 'plan$0$ROAD_BEGIN_END', 'points': [(330302.6263268062, 3604484.741736011, 0.0, 0.0, 0.0), (330289.0457915408, 3604457.255428256, 0.0, 0.0, 0.0)], 'meta': {}}, {'type': 'LWPOLYLINE', 'layer': 'plan$0$ROAD_BEGIN_END', 'points': [(202736.133928047, 3651214.961762448, 0.0, 0.0, 0.0), (202879.24588544, 3651131.538330482, 0.0, 0.0, 0.0)], 'meta': {}}, {'type': 'LWPOLYLINE', 'layer': 'plan$0$ROAD_BEGIN_END', 'points': [(216184.93499785, 3624900.168223931, 0.0, 0.0, 0.0), (216368.9816347847, 3624902.16592709, 0.0, 0.0, 0.0)], 'meta':

In [4]:
import pandas as pd

# Example: Converting to DataFrame
df = pd.DataFrame(entities)
# Or optionally, save as JSON
import json
with open("entities.json", "w") as f:
    json.dump(entities, f, default=str)


In [5]:
import math
import numpy as np

def distance(point1, point2):
    return np.linalg.norm(np.array(point2) - np.array(point1))

def compute_curve_radius(arc_entity):
    # For ARC entity, radius is in meta
    return arc_entity["meta"].get("radius", None)

def compute_polyline_curve_radii(polyline_points):
    # For polyline curves, approximate radii at each vertex (3 consecutive points)
    radii = []
    pts = polyline_points
    for i in range(1, len(pts) - 1):
        p1, p2, p3 = np.array(pts[i-1][:2]), np.array(pts[i][:2]), np.array(pts[i+1][:2])
        # Calculate radius using circumcircle formula for 3 points in 2D
        a = np.linalg.norm(p2 - p3)
        b = np.linalg.norm(p1 - p3)
        c = np.linalg.norm(p1 - p2)
        s = (a + b + c) / 2
        area = max(s*(s - a)*(s - b)*(s - c), 0)**0.5
        if area == 0:
            radius = float('inf')  # Straight line, infinite radius
        else:
            radius = (a*b*c) / (4 * area)
        radii.append(radius)
    return radii

def compute_segment_length(points):
    total_length = 0.0
    for i in range(len(points) - 1):
        total_length += distance(points[i][:2], points[i+1][:2])
    return total_length


In [6]:
# Example IRC parameters for demonstration (replace with actual IRC standard values)
IRC_MIN_CURVE_RADIUS = 120.0  # meters, example minimum horizontal curve radius
IRC_MIN_LANE_WIDTH = 3.5      # meters

def verify_min_curve_radius(curve_radius, min_radius=IRC_MIN_CURVE_RADIUS):
    if curve_radius is None:
        return False, "Curve radius not defined"
    if curve_radius < min_radius:
        return False, f"Curve radius {curve_radius:.2f} m is less than IRC minimum {min_radius} m"
    return True, "Curve radius compliant"

def verify_lane_width(lane_width, min_width=IRC_MIN_LANE_WIDTH):
    if lane_width < min_width:
        return False, f"Lane width {lane_width:.2f} m is less than IRC minimum {min_width} m"
    return True, "Lane width compliant"


In [7]:
def layer_to_element_type(layer_name):
    # A simple example mapping; extend with your layer naming conventions
    if "ROAD_BEGIN_END" in layer_name:
        return "road_edge"
    elif "Bariyer" in layer_name.lower() or "barrier" in layer_name.lower():
        return "safety_barrier"
    elif "DIM" in layer_name:
        return "dimension"
    else:
        return "other"


In [8]:
def verify_entities(entities):
    results = []
    for ent in entities:
        element_type = layer_to_element_type(ent["layer"])

        # Example: Check curve radius for ARC and curved polylines
        if ent["type"] == "ARC":
            radius = compute_curve_radius(ent)
            passed, msg = verify_min_curve_radius(radius)
            results.append({"entity": ent, "check": "curve_radius", "passed": passed, "message": msg})
        elif ent["type"] == "LWPOLYLINE":
            radii = compute_polyline_curve_radii(ent["points"])
            for r in radii:
                passed, msg = verify_min_curve_radius(r)
                results.append({"entity": ent, "check": "curve_radius", "passed": passed, "message": msg})
        # Add other checks like lane width, slope, safety barrier placement here using geometry and layer info
        
        # Placeholder for lane width check, assuming lane width stored or computed:
        # if element_type == "road_edge":
        #   lane_width = some_computation(ent)
        #   passed, msg = verify_lane_width(lane_width)
        #   results.append({"entity": ent, "check": "lane_width", "passed": passed, "message": msg})
        
    return results


In [9]:
def print_verification_report(results):
    for i, res in enumerate(results, 1):
        ent = res["entity"]
        print(f"{i}. Entity Type: {ent['type']} Layer: {ent['layer']}")
        print(f"   Check: {res['check']}  --> {'PASS' if res['passed'] else 'FAIL'}")
        print(f"   Message: {res['message']}")
        print()


In [10]:
verification_results = verify_entities(entities)

In [11]:
print_verification_report(verification_results)

1. Entity Type: LWPOLYLINE Layer: Bariyer orta
   Check: curve_radius  --> PASS
   Message: Curve radius compliant

2. Entity Type: LWPOLYLINE Layer: plan$0$ROAD_BEGIN_END
   Check: curve_radius  --> FAIL
   Message: Curve radius 82.69 m is less than IRC minimum 120.0 m

3. Entity Type: LWPOLYLINE Layer: plan$0$ROAD_BEGIN_END
   Check: curve_radius  --> FAIL
   Message: Curve radius 74.10 m is less than IRC minimum 120.0 m

4. Entity Type: LWPOLYLINE Layer: plan$0$ROAD_BEGIN_END
   Check: curve_radius  --> PASS
   Message: Curve radius compliant

5. Entity Type: LWPOLYLINE Layer: plan$0$ROAD_BEGIN_END
   Check: curve_radius  --> PASS
   Message: Curve radius compliant

6. Entity Type: LWPOLYLINE Layer: plan$0$ROAD_BEGIN_END
   Check: curve_radius  --> PASS
   Message: Curve radius compliant

7. Entity Type: LWPOLYLINE Layer: plan$0$ROAD_BEGIN_END
   Check: curve_radius  --> PASS
   Message: Curve radius compliant

8. Entity Type: LWPOLYLINE Layer: plan$0$ROAD_BEGIN_END
   Check: curve_

In [12]:
import ezdxf

def export_verification_to_dxf(results, output_path="result.dxf"):
    # Create a new DXF document with version R2010 (AutoCAD 2010)
    doc = ezdxf.new("R2010")
    msp = doc.modelspace()

    for result in results:
        entity = result["entity"]
        passed = result["passed"]
        # Use color 3 (green) if passed, else 1 (red)
        color = 3 if passed else 1

        if entity["type"] == "LINE":
            start_point, end_point = entity["points"]
            msp.add_line(start_point, end_point, dxfattribs={"color": color})

        elif entity["type"] == "LWPOLYLINE":
            # Convert 3D points to 2D by taking only (x, y)
            points_2d = [(pt[0], pt[1]) for pt in entity["points"]]
            msp.add_lwpolyline(points_2d, dxfattribs={"color": color})

        elif entity["type"] == "ARC":
            center = entity["points"][0]
            radius = entity["meta"]["radius"]
            # Add a full circle arc (0 to 360 degrees)
            msp.add_arc(center, radius, start_angle=0, end_angle=360, dxfattribs={"color": color})

        # Add handling for other entity types as needed

    # Save the DXF to specified output path
    doc.saveas(output_path)


In [13]:
try:
    export_verification_to_dxf(verification_results, output_path="result.dxf")
except Exception as e:
    print(f"Failed to export DXF: {e}")

In [14]:
import ezdxf
from ezdxf.addons import Importer

# Load the first DXF file
doc1 = ezdxf.readfile('/kaggle/input/intersection-one/59.Intersection drawings of highways Autocad File DWG.dxf')
msp1 = doc1.modelspace()

# Load the second DXF file
doc2 = ezdxf.readfile('/kaggle/working/result.dxf')
msp2 = doc2.modelspace()

# Create a new DXF document
merged_doc = ezdxf.new("R2010")
merged_msp = merged_doc.modelspace()

# Use Importer to bring in entities properly
importer1 = Importer(doc1, merged_doc)
importer1.import_modelspace()
importer1.finalize()

importer2 = Importer(doc2, merged_doc)
importer2.import_modelspace()
importer2.finalize()

# Save the combined DXF
output_file = "/kaggle/working/final_2.dxf"
merged_doc.saveas(output_file)

print(f"Merged DXF saved as {output_file}")


Merged DXF saved as /kaggle/working/final_2.dxf
